In [1]:
# %%capture
# #!pip install -U datasets
# !pip install transformers==4.18.0 nltk==3.2.4 numpy==1.21.6 datasets==2.1.0 pandas==1.3.5

In [2]:
import os
from copy import deepcopy
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from datasets import load_dataset, set_caching_enabled
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer, AutoFeatureExtractor,
    AutoModel, AutoConfig,            
    TrainingArguments, Trainer,
    logging
)
import warnings 
warnings.filterwarnings('ignore')

import nltk
nltk.download('wordnet')
import nltk
import subprocess

try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

from nltk.corpus import wordnet

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /kaggle/working/...


Archive:  /kaggle/working/corpora/wordnet.zip
   creating: /kaggle/working/corpora/wordnet/
  inflating: /kaggle/working/corpora/wordnet/lexnames  
  inflating: /kaggle/working/corpora/wordnet/data.verb  
  inflating: /kaggle/working/corpora/wordnet/index.adv  
  inflating: /kaggle/working/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/corpora/wordnet/index.verb  
  inflating: /kaggle/working/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/corpora/wordnet/data.adj  
  inflating: /kaggle/working/corpora/wordnet/index.adj  
  inflating: /kaggle/working/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/corpora/wordnet/noun.exc  
  inflating: /kaggle/working/corpora/wordnet/verb.exc  
  inflating: /kaggle/working/corpora/wordnet/README  
  inflating: /kaggle/working/corpora/wordnet/index.sense  
  inflating: /kaggle/working/corpora/wordnet/data.noun  


  inflating: /kaggle/working/corpora/wordnet/data.adv  
  inflating: /kaggle/working/corpora/wordnet/index.noun  
  inflating: /kaggle/working/corpora/wordnet/adj.exc  


In [3]:
os.environ['HF_HOME'] = os.path.join(".", "cache")

set_caching_enabled(False)
logging.set_verbosity_error()

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))


cuda
Tesla P100-PCIE-16GB


# Load the dataset

In [5]:
import pandas as pd
from datasets import Dataset, DatasetDict

train_df = pd.read_csv("/kaggle/input/vr-image-qna/data_new.csv")
test_df = pd.read_csv("/kaggle/input/vr-image-qna/data_val_new.csv")

# This converts pandas DataFrames to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Creating a DatasetDict
dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

with open("/kaggle/input/vr-image-qna/answer_space_unsampled2.txt") as f:
    answer_space = f.read().splitlines()

def process_examples(examples):
    labels = []
    count1 = 0
    count2 = 0
    for ans in examples['answer']:
        cleaned_answer = ans.replace(" ", "").split(",")[0]
        try:
            index = answer_space.index(ans)
            labels.append(index)
        except ValueError:
            # Handle unknown cases here, for example:
            labels.append(-1)  # Assign a default label for unknown cases
        
    return {'label': labels}


dataset = dataset.map(
    process_examples,
    batched=True
)

dataset

  0%|          | 0/111 [00:00<?, ?ba/s]

  0%|          | 0/54 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'imageId', 'questionId', 'questionType', 'answerType', 'kaggle_image_path', 'label'],
        num_rows: 110245
    })
    test: Dataset({
        features: ['question', 'answer', 'imageId', 'questionId', 'questionType', 'answerType', 'kaggle_image_path', 'label'],
        num_rows: 53564
    })
})

### Example of a question-image pair

Question:	 What bird is in the photo?
Answer:		 cardinal (Label: 6872)


'cardinal'

In [6]:
import IPython.display as display

def show_example(train=True, idx=None):
    dataset_type = "train" if train else "test"
    data = dataset[dataset_type]

    if idx is None:
        idx = np.random.randint(len(data))

    image_path =  data[idx]["kaggle_image_path"]
    image = Image.open(image_path)
    display.display(image)

    question = data[idx]["question"]
    answer = data[idx]["answer"]
    label = data[idx]["label"]

    print(f"Question:\t {question}")
    print(f"Answer:\t\t {answer} (Label: {label})")

    return answer
show_example()

## Generate a Multimodal Collator for the Dataset

This collator is designed to be utilized within the Trainer() to seamlessly construct the Dataloader from the dataset, streamlining the input pipeline to the model.

The collator's primary function is to handle both textual (question) and image data. It processes the question text, tokenizing it and generating attention masks. Simultaneously, it featurizes the image, essentially encapsulating its pixel values. These processed inputs are then fed into our multimodal transformer model, facilitating the question-answering process.

In [7]:
@dataclass
class MultimodalCollator:
    tokenizer: AutoTokenizer
    preprocessor: AutoFeatureExtractor
    
    def tokenize_text(self, texts: List[str]) -> Dict[str, torch.Tensor]:
        encoded_text = self.tokenizer(
            text=texts,
            padding='longest',
            max_length=24,
            truncation=True,
            return_tensors='pt',
            return_token_type_ids=True,
            return_attention_mask=True,
        )
        return {
            "input_ids": encoded_text['input_ids'].squeeze(),
            "token_type_ids": encoded_text['token_type_ids'].squeeze(),
            "attention_mask": encoded_text['attention_mask'].squeeze(),
        }
    
    def preprocess_images(self, images: List[str]):

        processed_images = self.preprocessor(
            images=[Image.open(image_id).convert('RGB') for image_id in images],
            return_tensors="pt",
        )
        return {
            "pixel_values": processed_images['pixel_values'].squeeze(),
        }
            
    def __call__(self, raw_batch_dict) -> Dict[str, torch.Tensor]:
        return {
            **self.tokenize_text(
                raw_batch_dict['question']
                if isinstance(raw_batch_dict, dict) else
                [i['question'] for i in raw_batch_dict]
            ),
            **self.preprocess_images(
                raw_batch_dict['kaggle_image_path']
                if isinstance(raw_batch_dict, dict) else
                [i['kaggle_image_path'] for i in raw_batch_dict]
            ),
            'labels': torch.tensor(
                raw_batch_dict['label']
                if isinstance(raw_batch_dict, dict) else
                [i['label'] for i in raw_batch_dict],
                dtype=torch.int64
            ),
        }


In [8]:
class MultimodalVQAModel(nn.Module):
    def __init__(
        self,
        num_labels: int = len(answer_space),
        intermediate_dim: int = 512,
        pretrained_text_name: str = 'bert-base-uncased',
        pretrained_image_name: str = 'google/vit-base-patch16-224-in21k'
    ):

        super(MultimodalVQAModel, self).__init__()
        self.num_labels = num_labels
        self.pretrained_text_name = pretrained_text_name
        self.pretrained_image_name = pretrained_image_name
        
        # Text and image encoders
        
        self.text_encoder = AutoModel.from_pretrained(self.pretrained_text_name)
        self.image_encoder = AutoModel.from_pretrained(self.pretrained_image_name)
        
        self.fusion = nn.Sequential(
            nn.Linear(self.text_encoder.config.hidden_size + self.image_encoder.config.hidden_size, intermediate_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
        )
        
        self.classifier = nn.Linear(intermediate_dim, self.num_labels)
        
        self.criterion = nn.CrossEntropyLoss()

    def forward(
        self,
        input_ids: torch.LongTensor,
        pixel_values: torch.FloatTensor,
        attention_mask: Optional[torch.LongTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        labels: Optional[torch.LongTensor] = None
    ):

        encoded_text = self.text_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True,
        )
        
        encoded_image = self.image_encoder(
            pixel_values=pixel_values,
            return_dict=True,
        )
        
        fused_output = self.fusion(
            torch.cat(
                [
                    encoded_text['pooler_output'],
                    encoded_image['pooler_output'],
                ],
                dim=1
            )
        )
        
        logits = self.classifier(fused_output)
        
        out = {"logits": logits}
        if labels is not None:
            loss = self.criterion(logits, labels)
            out["loss"] = loss
        
        return out


In [9]:
def create_multimodal_vqa_collator_and_model(text_encoder='bert-base-uncased', image_encoder='google/vit-base-patch16-224-in21k'):

    tokenizer = AutoTokenizer.from_pretrained(text_encoder)
    preprocessor = AutoFeatureExtractor.from_pretrained(image_encoder)
    
    multimodal_collator = MultimodalCollator(
        tokenizer=tokenizer,
        preprocessor=preprocessor,
    )

    multimodal_model = MultimodalVQAModel(
        pretrained_text_name=text_encoder,
        pretrained_image_name=image_encoder
    ).to(device)

    return multimodal_collator, multimodal_model


In [10]:
def wup_measure(a, b, similarity_threshold=0.925):
    def get_semantic_field(word):
        weight = 1.0
        semantic_field = wordnet.synsets(word, pos=wordnet.NOUN)
        return semantic_field, weight

    def get_stem_word(word):
        weight = 1.0
        return word, weight

    global_weight = 1.0

    a, global_weight_a = get_stem_word(a)
    b, global_weight_b = get_stem_word(b)
    global_weight = min(global_weight_a, global_weight_b)

    if a == b:
        return 1.0 * global_weight

    if a == "" or b == "":
        return 0

    interp_a, weight_a = get_semantic_field(a)
    interp_b, weight_b = get_semantic_field(b)

    if interp_a == [] or interp_b == []:
        return 0

    global_max = 0.0
    for x in interp_a:
        for y in interp_b:
            local_score = x.wup_similarity(y)
            if local_score > global_max:
                global_max = local_score

    if global_max < similarity_threshold:
        interp_weight = 0.1
    else:
        interp_weight = 1.0

    final_score = global_max * weight_a * weight_b * interp_weight * global_weight
    return final_score

def batch_wup_measure(labels, preds):
    wup_scores = [wup_measure(answer_space[label], answer_space[pred]) for label, pred in zip(labels, preds)]
    return np.mean(wup_scores)


In [11]:
labels = np.random.randint(len(answer_space), size=5)
preds = np.random.randint(len(answer_space), size=5)

def showAnswers(ids):
    print([answer_space[id] for id in ids])

showAnswers(labels)
showAnswers(preds)

print("Predictions vs Labels: ", batch_wup_measure(labels, preds))
print("Labels vs Labels: ", batch_wup_measure(labels, labels))

['sweet juicy navels', 'octagon and rectangle', 'no blender', 'creme brulee', 'roses and tulips']
['instantdecisifcom', 'tonawanda', 'parade horse', 'fishing hat', '1042']


Predictions vs Labels:  0.0
Labels vs Labels:  1.0


In [12]:
def compute_metrics(eval_tuple: Tuple[np.ndarray, np.ndarray]) -> Dict[str, float]:
    logits, labels = eval_tuple
    preds = logits.argmax(axis=-1)
    metrics = {
        "wups": batch_wup_measure(labels, preds),
        "acc": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average='weighted'),
        "precision": precision_score(labels, preds, average='weighted'),  # Added average='macro'
        "recall": recall_score(labels, preds, average='weighted') 
    }

    return metrics


# Train the model and evaluate

In [13]:
args = TrainingArguments(
    output_dir="/kaggle/working/checkpoint/",            
    seed=12345,                         
    evaluation_strategy="steps",        
    eval_steps=100,                     
    logging_strategy="steps",           
    logging_steps=100,                  
    save_strategy="steps",              
    save_steps=100,                     
    save_total_limit=3,                  
    metric_for_best_model='wups',       
    per_device_train_batch_size=64,     
    per_device_eval_batch_size=64,      
    remove_unused_columns=False,        
    num_train_epochs=5,                 
    fp16=True,                          
    dataloader_num_workers=8,           
    load_best_model_at_end=True,        
)
    



In [14]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_key = user_secrets.get_secret("wandb2")
os.environ["WANDB_API_KEY"] = wandb_key
wandb.login(key = wandb_key)
!wandb status

wandb: Currently logged in as: achintyaharsha03 (achiorg). Use `wandb login --relogin` to force relogin


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Current Settings
{
  "_extra_http_headers": null,
  "_proxies": null,
  "api_key": null,
  "base_url": "https://api.wandb.ai",
  "entity": null,
  "git_remote": "origin",
  "ignore_globs": [],
  "project": null,
  "root_dir": null,
  "section": "default"
}


In [15]:
def create_and_train_model(dataset, args, text_model='bert-base-uncased', image_model='google/vit-base-patch16-224-in21k', multimodal_model='bert_vit'):
    collator, model = create_multimodal_vqa_collator_and_model(text_model, image_model)
    
    multi_args = deepcopy(args)
    multi_args.output_dir = os.path.join("/kaggle/working/checkpoint/", multimodal_model)
    print(multi_args.output_dir)
    multi_trainer = Trainer(
        model,
        multi_args,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        data_collator=collator,
        compute_metrics=compute_metrics
    )
    
    train_multi_metrics = multi_trainer.train()
    eval_multi_metrics = multi_trainer.evaluate()
    
    return collator, model, train_multi_metrics, eval_multi_metrics, multi_trainer


In [4]:
collator, model, train_multi_metrics, eval_multi_metrics, trainer = create_and_train_model(dataset, args)

/kaggle/working/checkpoint/bert_vit


wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Tracking run with wandb version 0.15.12


wandb: Run data is saved locally in /kaggle/working/wandb/run-20240517_110316-kvlxvb3w
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run /kaggle/working/checkpoint/


wandb: ⭐️ View project at https://wandb.ai/achiorg/huggingface


wandb: 🚀 View run at https://wandb.ai/achiorg/huggingface/runs/kvlxvb3w


{'loss': 7.4729, 'learning_rate': 4.942542077771329e-05, 'epoch': 0.06}
{'eval_loss': 5.5115180015563965, 'eval_wups': 0.207371295722818, 'eval_acc': 0.18962362780972294, 'eval_f1': 0.001208706361120421 'eval_precision': 0.01 'eval_recall': 0.0006432267965945583 'eval_runtime': 503.431, 'eval_samples_per_second': 106.398, 'eval_steps_per_second': 1.663, 'epoch': 0.06}
{'loss': 5.1886, 'learning_rate': 4.884503772489843e-05, 'epoch': 0.12}
{'eval_loss': 4.658883094787598, 'eval_wups': 0.26251919489714565, 'eval_acc': 0.23769696064520948, 'eval_f1': 0.0028830910043566925 'eval_precision': 0.010921568627450981 'eval_recall': 0.001660748916282978 'eval_runtime': 533.1253, 'eval_samples_per_second': 100.472, 'eval_steps_per_second': 1.57, 'epoch': 0.12}
{'loss': 4.7799, 'learning_rate': 4.826465467208358e-05, 'epoch': 0.17}
{'eval_loss': 4.418029308319092, 'eval_wups': 0.2617689608897943, 'eval_acc': 0.2358113658427302, 'eval_f1': 0.0029086100032640226 'eval_precision': 0.01184313725490196 

In [1]:
def count_trainable_parameters(model):
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Number of trainable parameters: {:,}".format(num_params))
count_trainable_parameters(model)

Number of trainable parameters: 211,705,748
